### Practical Decorators by Reuven M Lerner

In [1]:
"""
@mydeco
def add(a,b):
    return a+b 
"""
# the above code has 3 callables -- > something we can execute
# 1st --> add
# 2nd is @mydeco
# 3rd which is return from mydeco(add)


'\n@mydeco\ndef ad(a,b):\n    return a+b \n'

In [4]:
## defining a decorator

def mydeco(func):
    def wrapper(*args, **kwargs):
        return "{} !!!".format(func(*args, **kwargs))
    return wrapper

""" How is it that the internal function has access to func. Func is a local variable in the external 
    function . This is achieved by Python Scoping Local and Closing Builtin . It first checks for 
    func in local def but if it is not found then it searches in the external def.
"""
        
@mydeco
def add(a,b):
    return a+b 

@mydeco
def square(a):
    return a**2

In [5]:
add(3,4)
square(4)

'16 !!!'

#  

### Decorators practical examples :

#  

### Example 1.
###### Timing a function

In [42]:
import time
def logtime(func):
    
    def wrapper(*args,**kawrgs):
        start = time.time()
        rv = func(*args,**kawrgs)
        end = time.time()
        
        total_time = end-start
        with open("logtime.txt","a+") as f:
            f.write("Time taken by function: {} with arguments : {!r} is >>>>> {}\n".format(func.__name__,args,total_time))
        
        return rv
    return wrapper

@logtime
def genIterate(n,m=5):
    return [i*i for i in range(n*m)]
        


rv = genIterate(10000,6)

#  

###  Example 2
##### Once per minute : Raise an exception if we try to run a function once per minute
##### This funciton is very expensive in terms of resource hence we dont want it to run more than once in 1 min

In [67]:
def oncePerMin(func):
    last_invoked = 0
    def wrapper(*args,**kwargs):
        nonlocal last_invoked
        
        elapsed_time = time.time() - last_invoked
        if elapsed_time<60:
            raise Exception("Only {} has elapsed , wait for atleast 1 min".format(elapsed_time))
        last_invoked = time.time()
        
        return func(*args,**kwargs)
        
    return wrapper



""" Lets generalize it to once per m """
def oncePerM(m):
    
    def middle(func):
        last_invoked = 0
        def wrapper(*args,**kwargs):
            nonlocal last_invoked

            elapsed_time = time.time() - last_invoked
            if elapsed_time<m:
                raise Exception("Only {} has elapsed , wait for atleast {} sec".format(elapsed_time,m))
            last_invoked = time.time()

            return func(*args,**kwargs)

        return wrapper
    return middle



@oncePerMin
def genIterate(n):
    [i*i for i in range(n)]


@oncePerM(5)
def genIteratePerM(n):
    [i*i for i in range(n)]


In [73]:
genIteratePerM(10)

#  
#  

###  Example 3
#### Memoization 
##### Cache results of function call so we dont have to call them again and again

In [99]:
def memoize(func):
    
    cache = {}
    """ We dont use nonlocal here because we are not assigning to the
        variable we are just updating the variable using cache['something']
        Had we used cache = 'something' then we had to use nonlocal
    """
    def wrapper(*args,**kwargs):
        if args not in cache:
            print("Caching NEW values for function : {} with arguments {}".format(func.__name__,args))
            cache[args] = func(*args,**kwargs)
        else:
            print("Using OLD values for function : {} with arguments {}".format(func.__name__,args))
        return cache[args]
    return wrapper

@memoize
def fib(n):
    a,b=0,1
    for i in range(n):
        a,b=b,a+b
    return a

@memoize
def add(a,b):
    return a+b

add(2,3)
add(2,3)

Caching NEW values for function : add with arguments (2, 3)
Using OLD values for function : add with arguments (2, 3)


5

### Example 4. Using picking for caching

In [95]:

""" But what if *args are non hashable 
    And what about **kwargs . 
    
    Here comes pickle to the rescue . 
    bcz -- > It converts a python datastructure to strings and bytestrings.
    Strings and bytestrings -- > These are hashable
    And just about anything can be pickled
    So use a tuple of your bytestrings as your dict keys and you will be fine 
    for most purposes
    If all this doesnt work you can always call the function
    
"""
import pickle

def memoizePickled(func):
    
    cache = {}
    """ We dont use nonlocal here because we are not assigning to the
        variable we are just updating the variable using cache['something']
        Had we used cache = 'something' then we had to use nonlocal
    """
    def wrapper(*args,**kwargs):
        t = (pickle.dumps(args),pickle.dumps(kwargs))
        if t not in cache:
            print("Caching NEW values for function : {} with arguments {}".format(func.__name__,args))
            cache[t] = func(*args,**kwargs)
        else:
            print("Using OLD values for function : {} with arguments {}".format(func.__name__,args))
        return cache[t]
    return wrapper


@memoize
def add(*args):
    sum = 0
    for i in args:
        sum+=i
        
    return sum



Caching NEW values for function : add with arguments (2, 3, 4, 5)


14

In [98]:
add(2,3,4,5)
add(2,3,5,4)

Using OLD values for function : add with arguments (2, 3, 4, 5)
Using OLD values for function : add with arguments (2, 3, 5, 4)


14

#  
#  
### Example 5. Attributes 

##### Give many objects the same attributes, but without inheritance . 
##### For Example methods are attributes of class. What if we want to have same attributes for all the classes

In [122]:
""" Now inheritance is cool when we have similar classes but if we want to have
    the same atributes in other distinct classes then we can use this decorator
    
    >> We want to have a bunch of attributes consistently set across all the classes
    >> These classes are not related so we wont use inheritance
    >> And we dont want multiple inheritance ( it is terrible)
"""
## We create a fancy __repr__ of a class now. 
## So we need to create a repr function of a class now
## but using a decorator instead of dunder __repr__
from datetime import datetime

def fancy_repr(self):
    return "I am a repr of {} with vars {}".format(type(self).__name__,vars(self))


def better_repr(cls):
    cls.__repr__ = fancy_repr
    def wrapper(*args,**kwargs):
        obj = cls(*args,**kwargs)
        return obj
    
    return wrapper


def better_repr_simp(cls):
    cls.__repr__ = fancy_repr
    return cls


def object_bday(cls):
    def wrapper(*args,**kwargs):
        obj = cls(*args,**kwargs)
        obj._birthday = time.time()
        
        return obj
    return wrapper


def repr_with_birthday(cls):
    cls.__repr__ = fancy_repr
    
    def wrapper(*args,**kwargs):
        obj = cls(*args,**kwargs)
        obj._birthday = datetime.strftime(datetime.today(),"%Y-%m-%d %H:%M:%S")
        
        return obj
    return wrapper
    
        
@object_bday
@better_repr_simp
class Foo:
    def __init__(self,x,y):
        self.x =x
        self.y =y
        
@repr_with_birthday    
class Foobar:
    def __init__(self,x,y):
        self.x =x
        self.y =y    
        
        
f = Foo(2,3)
fb = Foobar(5,7)

print(f)
print(fb)
    




I am a repr of Foo with vars {'x': 2, 'y': 3, '_birthday': 1611478602.3323314}
I am a repr of Foobar with vars {'x': 5, 'y': 7, '_birthday': '2021-01-24 14:26:42'}


#### Conclusions :

##### Decorators let you DRY ( Dont Repeat Yourself)  up your callables : Dont Repeat Yourself
##### Underatanding the number of callables are involved makes it easier to see what problems can be solved and how
##### Decorators make it dramaticaly easy to do many things
##### Many of it depends on the fact that in python , callables ( functions and classes) are objects like any other and can be passed and returned easily